In [ ]:
%run operator.ipynb

In [ ]:
'''
alpha = group_rank(fn_assets_fair_val_a, industry) > 0.5 ? (-group_rank(fnd2_ebitdm, industry) - group_rank(fnd2_ebitfr, industry)) * 2 : (-group_rank(fnd2_ebitdm, industry) - group_rank(fnd2_ebitfr, industry))

這個alpha假設高資產公平價值但相對較低EBIT的公司可能是過去幾年中投資增長並且未來增長空間較大的公司。

將國內和國外EBIT分別排名並按行業分組。這是因為不同行業可能在國內和國外EBIT之間有不同的分配比例。對於資產公平價值較低的公司 設置較低的alpha信號。

- **提升Alpha的建議 HINT TO IMPROVE THE ALPHA 如果將比較已經更大規模的公司 TOP500 之間的公司 那麼這個alpha是否會發揮良好 這是因為太小的公司可能認為這個指標太冒險 因為這是低現金流的一個跡象。
'''
temp = 0
temp_alpha = ''
temp_i = 0
for i in range(6, 9): #1represent 0.1, 2 represent 0.2, 3 represent 0.3, 4 represent 0.4alpha = f'-group_rank(fnd2_ebitdm, industry) - group_rank(fnd2_ebitfr, industry);\ngroup_rank(fn_assets_fair_val_a, industry) > {i/10} ? alpha * 2 : alpha'
    alpha = f'group_rank(fn_assets_fair_val_a, industry) > {i/10} ? (-group_rank(fnd2_ebitdm, industry) - group_rank(fnd2_ebitfr, industry)) * 2 : (-group_rank(fnd2_ebitdm, industry) - group_rank(fnd2_ebitfr, industry))'
    simulation_data = {
        "type":"REGULAR",
        "settings":{
            "nanHandling":"OFF",
            "instrumentType":"EQUITY",
            "delay":1,
            "universe":"TOP500",
            "truncation":0.08,
            "unitHandling":"VERIFY",
            "pasteurization":"ON",
            "region":"USA",
            "language":"FASTEXPR",
            "decay":0,
            "neutralization":"SUBINDUSTRY",
            "visualization":False,
        },
        'regular': alpha
    }
    simulation_response = sess.post(SIMULATION_ENDPOINT, json=simulation_data)
    simulation_progress_url = simulation_response.headers['Location']
    finished = False
    while True:
        simulation_progress = sess.get(simulation_progress_url)
        if simulation_progress.headers.get('Retry-After',0) == 0:
            break
        print('Sleeping for'+simulation_progress.headers['Retry-After'],'seconds')
        sleep.sleep(float(simulation_progress.headers['Retry-After']))
    print('Alpha done running')
    alpha_id = simulation_progress.json()["alpha"]
    alpha = sess.get(f'{ALPHA_ENDPOINT}/{alpha_id}')
    simulation_progress.json()
    alpha_resp = sess.get(ALPHA_ENDPOINT+'/'+alpha_id).json()
    print(alpha_resp['is']['sharpe'])
    if alpha_resp['is']['sharpe'] > temp:
        temp = alpha_resp['is']['sharpe']
        temp_alpha = alpha
        print(temp)
        temp_i = i
    print(i)
print('the best alpha is', temp_i)

In [ ]:
#接續上一個cell 現在我要更動fn_assets_fair_val_a
#其替代品有 fn_assets_fair_val_a, fn_assets_fair_val_q, fn_assets_fair_val_l1_a, fn_assets_fair_val_l1q, fn_assets_fair_val_l2a, fn_assets_fair_val_l2q, fn_assets_fair_val_l3a, fn_assets_fair_val_l3q
#我要找出最好的fn_assets_fair_val_a的替代品 
#都設定為0.7
#只變動fn_assets_fair_val_a
#所以應該要設定一個array儲存fn_assets_fair_val_a, fn_assets_fair_val_q, fn_assets_fair_val_l1a, fn_assets_fair_val_l1q, fn_assets_fair_val_l2a, fn_assets_fair_val_l2q, fn_assets_fair_val_l3a, fn_assets_fair_val_l3q
#跑過fn_assets_fair_val_a 到 fn_assets_fair_val_l3q
temp = 0
temp_alpha = ''
temp_i = 0
chars = ['a', 'q', 'l1_a', 'l1_q', 'l2_a', 'l2_q', 'l3_a', 'l3_q']
for i in range(0, 7):
    alpha = f'group_rank(fn_assets_fair_val_{chars[i]}, industry) > 0.7 ? (-group_rank(fnd2_ebitdm, industry) - group_rank(fnd2_ebitfr, industry)) * 2 : (-group_rank(fnd2_ebitdm, industry) - group_rank(fnd2_ebitfr, industry))'
    simulation_data = {
        "type":"REGULAR",
        "settings":{
            "nanHandling":"OFF",
            "instrumentType":"EQUITY",
            "delay":1,
            "universe":"TOP500",
            "truncation":0.08,
            "unitHandling":"VERIFY",
            "pasteurization":"ON",
            "region":"USA",
            "language":"FASTEXPR",
            "decay":0,
            "neutralization":"SUBINDUSTRY",
            "visualization":False,
        },
        'regular': alpha
    }
    simulation_response = sess.post(SIMULATION_ENDPOINT, json=simulation_data)
    simulation_progress_url = simulation_response.headers['Location']
    finished = False
    while True:
        simulation_progress = sess.get(simulation_progress_url)
        if simulation_progress.headers.get('Retry-After',0) == 0:
            break
        print('Sleeping for'+simulation_progress.headers['Retry-After'],'seconds')
        sleep.sleep(float(simulation_progress.headers['Retry-After']))
    print('Alpha done running')
    alpha_id = simulation_progress.json()["alpha"]
    alpha = sess.get(f'{ALPHA_ENDPOINT}/{alpha_id}')
    simulation_progress.json()
    alpha_resp = sess.get(ALPHA_ENDPOINT+'/'+alpha_id).json()
    print(alpha_resp['is']['sharpe'])
    if alpha_resp['is']['sharpe'] > temp:
        temp = alpha_resp['is']['sharpe']
        temp_alpha = alpha
        print(temp)
        temp_i = i
    print(i)
print('the best alpha is', temp_i)